In [1]:
# _____________ read from SQLite and write to Azure
# connect to current source SQLite
import sqlite3
# import os
# import datetime
# import tqdm
# import glob
import pandas as pd
conn = sqlite3.connect(r"D:\NMT\OneDrive\Viettravel Airline\Database\VTA_RM.db")
c = conn.cursor()

In [41]:
def delete_rows_SQL(table, file_mame):
    query = f"""
        DELETE FROM {table}
            WHERE file_name = '{file_mame}'
        """
    c.execute(query)

def delete_rows_cmd(row):
    delete_rows_SQL(row['table_name'], row['file_name'])


def file_name_modified(path:str):
    all_dir_files = glob.glob(path + '/*')
    files = pd.DataFrame()
    m_time = []

    for dir_file in all_dir_files:
        # file modification timestamp of a file
        f_m_time = os.path.getmtime(dir_file)
        # convert timestamp into DateTime object
        f_dt_m = datetime.datetime.fromtimestamp(f_m_time)
        m_time += [f_dt_m]

    files_name = pd.Series(all_dir_files).str.split('\\', expand=True).iloc[:, -1]
    table_name = pd.Series(all_dir_files).str.split('\\', expand=True).iloc[:, -2]
    return pd.DataFrame({'file_name':files_name, 'modified_time':m_time,
                         'table_name':table_name, 'dir_file':all_dir_files})

In [3]:
query = """
WITH confirm_pax AS(
    SELECT confirm, COUNT(*) AS pax
        FROM reservation
        GROUP BY confirm),
    confirm_amount AS(
    SELECT confirmation_num AS confirm, date_paid_lcl, SUM(rpt_amount) AS total_amount
        FROM payment_detail
        GROUP BY confirmation_num)
        
        
SELECT *
    FROM confirm_amount
    LEFT JOIN confirm_pax USING(confirm)
    WHERE date_paid_lcl >= '2022-09-01'

"""

pd.read_sql_query(query, conn).to_excel('aaa.xlsx', index=False)

In [5]:
query = """
WITH payment_agentcode AS(
    SELECT confirmation_num AS confirm, strftime('%Y-%m-%d' , DATE_PAID_LCL) AS Paid_Date, strftime('%H:%M:%S' , DATE_PAID_LCL) AS Paid_Time, RPT_AMOUNT,
            COALESCE(IATA_NUM, USER_ID, BOOKING_AGENT) AS ori_agent_code
        FROM payment_detail
        WHERE TRANS_STATUS = 'Approved'),
        confirm_pax AS(
    SELECT confirm, COUNT(*) AS pax
        FROM reservation
        GROUP BY confirm)

SELECT confirm, Paid_Date, rpt_amount, da.no AS Agent_code, channel, agent, pax
    FROM payment_agentcode pa
    LEFT JOIN dim_agent da
        ON pa.ori_agent_code = da.AG_Code
    LEFT JOIN confirm_pax USING(confirm)
    WHERE Paid_Date >= '2022-09-01'
"""

pd.read_sql_query(query, conn).to_excel('by_agent.xlsx', index=False)

In [6]:
query = """
WITH payment_agentcode AS(
    SELECT confirmation_num AS confirm, strftime('%Y-%m-%d' , DATE_PAID_LCL) AS Paid_Date, RPT_AMOUNT,
            COALESCE(IATA_NUM, USER_ID, BOOKING_AGENT) AS ori_agent_code
        FROM payment_detail
        WHERE TRANS_STATUS = 'Approved')
        
SELECT confirm, paid_date, rpt_amount, channel, agent
    FROM payment_agentcode
    LEFT JOIN dim_agent da
        ON ori_agent_code = da.AG_Code
    WHERE Paid_Date >= '2022-09-01'
"""

pd.read_sql_query(query, conn).to_excel('by_agent.xlsx', index=False)